In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import wntr
from pathlib import Path
from infrarisk.src.network_sim_models.interdependencies import *
from infrarisk.src.network_recovery import *

import math
import copy

In [4]:
network_dir= 'micropolis'

MAIN_DIR = Path('../..')
water_file = MAIN_DIR/'infrarisk/data/networks/{0}/water/water.inp'.format(network_dir)

step_size = 60
try:
    wn_original = wntr.network.WaterNetworkModel(water_file)
    wn_original.options.time.duration = 3600
    wn_original.options.time.report_timestep = 3600
    wn_original.options.time.hydraulic_timestep = 60
    wn_original.options.hydraulic.demand_model = 'DDA'
    
    print("Water network successfully loaded.")
except FileNotFoundError:
    print("Error: The water network file does not exist. No such file or directory: ", water_file)

Water network successfully loaded.


C:\Users\srijith\anaconda3\envs\redcar\lib\site-packages\wntr\epanet\io.py:2173: UserWarning: Not all curves were used in "..\..\infrarisk\data\networks\micropolis\water\water.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [26]:
wn = copy.deepcopy(wn_original)
components = ['W_PSC282', 'W_PMA535', 'W_PSC1054', 'W_PMA1043']
#components = ['W_PMA1050', 'W_PMA1118', 'W_PMA609', 'W_PMA608' , 'W_PMA937']
start_time = 6000
end_time = 22000

for component in components:
    wn = wntr.morph.split_pipe(wn,component,f"{component}_B",f"{component}_leak_node",)

    leak_node = wn.get_node(f"{component}_leak_node")
    leak_node.remove_leak(wn)
    leak_node.add_leak(wn,area=0.75*(math.pi*(wn.get_link(f"{component}_B").diameter)** 2)/4,
        start_time=start_time,
        end_time=end_time,
    )

    link_close_event(wn, component, end_time, "repairing")
    link_open_event(wn, component, end_time + 20000, "repairing")

In [27]:
while wn.options.time.duration <=  3600*72:
    #print("Simulation time: ", wn.options.time.duration,"; Hydraulic time step: ", wn.options.time.hydraulic_timestep, "; Report time step: ", wn.options.time.report_timestep)
    wn_sim = wntr.sim.WNTRSimulator(wn)
    wn_results = wn_sim.run_sim(convergence_error=True, solver_options={"MAXITER": 10000})
        
    #print('failed pipe flowrate: ', wn_results.link['flowrate'][components].values)
    # print('total leak demand: ', wn_results.node['leak_demand'][[f'{component}_leak_node' for component in components]].values)
    # print('tank head: ', wn_results.node['head']['W_T1'].values)
    # print('total pump flow rate: ', wn_results.link['flowrate'][wn.pump_name_list].values)
    
    sim_time = wn.options.time.duration
    node_results = wn_results.node["demand"].iloc[-1]

    water_supplied_at_t = sum([node_results[junc] for junc in wn.junction_name_list])
    base_demands_at_t = []
        
    for junc in wn.junction_name_list:
        base_demand = wn.get_node(junc).base_demand

        if base_demand != 0:
            pattern = (
                wn.get_node(junc)
                .demand_timeseries_list[0]
                .pattern.multipliers
            )
            pattern_size = len(pattern)
            pattern_interval = 24 / pattern_size
            pattern_index = math.floor(
                (((sim_time) / 3600) % 24) / pattern_interval
            )

            multiplier = pattern[pattern_index]
            base_demands_at_t.append(multiplier * base_demand)

    print("Time: ",wn.options.time.duration, "Supply: ", water_supplied_at_t, "Base demand: ", sum(base_demands_at_t))
    
    #print('***************')
    wn.options.time.duration += 3600

Time:  3600.0 Supply:  0.0360637615782266 Base demand:  0.0360637615782266
Time:  7200.0 Supply:  0.03755396575620661 Base demand:  0.03755396575620661
Time:  10800.0 Supply:  0.04200915624216557 Base demand:  0.04200915624216557
Time:  14400.0 Supply:  0.05251061062948877 Base demand:  0.05251061062948877
Time:  18000.0 Supply:  0.06422128205666872 Base demand:  0.06422128205666872
Time:  21600.0 Supply:  0.0730451064175469 Base demand:  0.0730451064175469
Time:  25200.0 Supply:  0.07611707696436859 Base demand:  0.07622419009773153
Time:  28800.0 Supply:  0.08280771753009859 Base demand:  0.082947786528521
Time:  32400.0 Supply:  0.08477132602594924 Base demand:  0.0849375511682685
Time:  36000.0 Supply:  0.08269113317624172 Base demand:  0.0828666816475733
Time:  39600.0 Supply:  0.08286007153028502 Base demand:  0.0830414558447773
Time:  43200.0 Supply:  0.08451370051342641 Base demand:  0.08451370051342641
Time:  46800.0 Supply:  0.08845564957064983 Base demand:  0.088455649570649

In [19]:
pattern_size = 12
sim_time = 7200
pattern_interval = 24 / pattern_size
pattern_index = math.floor(
    (((sim_time) / 3600) % 24) / pattern_interval
)
pattern_index

1

In [ ]:
wn_results.node['demand'].to_csv('results.csv')

In [ ]:
stop

In [ ]:
t = 86400
node_results = wn_results.node['demand'].iloc[-1]

water_supplied = sum([node_results[junc] for junc in wn.junction_name_list])

base_demands_at_t = []

for junc in wn.junction_name_list:
    base_demand = wn.get_node(junc).base_demand
    
    if base_demand != 0:
        demand = node_results[junc]
        pattern = wn.get_node(junc).demand_timeseries_list[0].pattern.multipliers
        #print(pattern)
        pattern_size = len(pattern)
        pattern_interval = 24/pattern_size
        
        pattern_index = math.ceil(((t/3600)%86400) / pattern_interval)
        multiplier = pattern[pattern_index-1]
        base_demands_at_t.append(multiplier*base_demand)
        

water_resmetric = water_supplied/sum(base_demands_at_t)
print(water_resmetric)

1.0


In [ ]:
wn.pump_name_list

['W_WP1', 'W_WP2', 'W_WP3', 'W_WP4', 'W_WP5', 'W_WP6', 'W_WP7', 'W_WP8']